
### Data Collection and Preprocessing
  - Collect historical energy consumption,holidays, pricing, weather, and population data.
  - Preprocess the data, handle missing values, and engineer features.

In [124]:
## Import Libraries 

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import io 
import os

from bs4 import BeautifulSoup


### Demand Data

In [159]:
## Getting all demand data from IESO website and putting into a data directory

# URL of the webpage containing the links to the CSV files
url = 'http://reports.ieso.ca/public/Demand/'

# Create a directory to store the downloaded files
os.makedirs('demand_data', exist_ok=True)

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find all anchor tags (<a>) that contain links to CSV files
links = soup.find_all('a', href=True)

# Extract the filenames from the href attributes of the anchor tags
csv_files = [link['href'] for link in links if link['href'].endswith('.csv')]

# Download each CSV file and save it in the "data" directory
for filename in csv_files:
    file_url = url + filename
    response = requests.get(file_url)
    with open(os.path.join('demand_data', filename), 'wb') as f:
        f.write(response.content)
        print(f"Downloaded {filename}")

Downloaded PUB_Demand.csv
Downloaded PUB_Demand_2002.csv
Downloaded PUB_Demand_2002_v1.csv
Downloaded PUB_Demand_2003.csv
Downloaded PUB_Demand_2003_v1.csv
Downloaded PUB_Demand_2004.csv
Downloaded PUB_Demand_2004_v1.csv
Downloaded PUB_Demand_2005.csv
Downloaded PUB_Demand_2005_v1.csv
Downloaded PUB_Demand_2006.csv
Downloaded PUB_Demand_2006_v1.csv
Downloaded PUB_Demand_2007.csv
Downloaded PUB_Demand_2007_v1.csv
Downloaded PUB_Demand_2008.csv
Downloaded PUB_Demand_2008_v1.csv
Downloaded PUB_Demand_2009.csv
Downloaded PUB_Demand_2009_v1.csv
Downloaded PUB_Demand_2010.csv
Downloaded PUB_Demand_2010_v1.csv
Downloaded PUB_Demand_2011.csv
Downloaded PUB_Demand_2011_v1.csv
Downloaded PUB_Demand_2012.csv
Downloaded PUB_Demand_2012_v1.csv
Downloaded PUB_Demand_2013.csv
Downloaded PUB_Demand_2013_v1.csv
Downloaded PUB_Demand_2014.csv
Downloaded PUB_Demand_2014_v1.csv
Downloaded PUB_Demand_2015.csv
Downloaded PUB_Demand_2015_v1.csv
Downloaded PUB_Demand_2016.csv
Downloaded PUB_Demand_2016_v1.csv

In [160]:
## Define a function to read the demand data from a CSV file and return a DataFrame in proper format
def read_csv(filename):
    # Read the CSV file into a DataFrame, skipping the first three rows
    df = pd.read_csv(filename, skiprows=[0, 1])
    # Set column names from the fourth row
    df.columns = df.iloc[0]
    # Drop the fourth row
    df = df.drop(index=0).reset_index(drop=True)
    return df

In [161]:
#Defining a function to combine all the CSV files into a single DataFrame

def combine_csvs(directory):
    """Combine all CSV files in a directory into a single DataFrame."""

# List to store DataFrames for each CSV file
    dfs = []

    # Iterate over each CSV file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            # Clean and read the CSV file into a DataFrame
            df = read_csv(file_path)
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    df = pd.concat(dfs, ignore_index=True)

    # Sort by the 'Date' column
    df = df.sort_values('Date')
    return df


In [162]:
## Define a function that takes a DataFrame and returns a new DataFrame with the 'Date' and 'Hour' columns combined into a single 'DateTime' column
def combine_date_hour(df):
    """Combine 'Date' and 'Hour' columns into a single 'DateTime' column."""
    # Convert 'Hour' to 24-hour format as integer and subtract 1
    df['Hour'] = (df['Hour'].astype(int) - 1) % 24  # Ensure hour values are within 0-23 range
    # Combine 'Date' and 'Hour' strings
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Hour'].astype(str) + ':00:00')
    # Drop 'Date' and 'Hour' columns
    df.drop(columns=['Date', 'Hour'], inplace=True)
    # # Sort DataFrame by the 'DateTime' column
    df.sort_values(by='DateTime', inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)
    return df

In [164]:
## Get the combined demand data, sort the date, and return the final dataframe 

# Combine all CSV files in the "data" directory into a single DataFrame
df_d = combine_csvs('demand_data')
# Combine the 'Date' and 'Hour' columns into a single 'DateTime' column
df_d  = combine_date_hour(df_d )
## set index to DateTime
df_d .set_index('DateTime', inplace=True)
## Sort the DataFrame by the 'DateTime' column
df_d .sort_values(by='DateTime', inplace=True)
# Remove duplicate rows
df_d = df_d.drop_duplicates()

# Display the first 50 rows of the DataFrame
df_d .head(50)

,Market Demand,Ontario Demand
DateTime,,
2002-05-01 00:00:00,14141,14137
2002-05-01 01:00:00,13876,13872
2002-05-01 02:00:00,13974,13820
2002-05-01 03:00:00,13898,13744
2002-05-01 04:00:00,14378,14224
2002-05-01 05:00:00,15408,15404
2002-05-01 06:00:00,17070,17066
2002-05-01 07:00:00,17644,17640
2002-05-01 08:00:00,17723,17719


### Pricing Data 






In [156]:
import requests
import os

# URL of the directory containing the CSV files
url = 'http://reports.ieso.ca/public/PriceHOEPPredispOR/'

# Create a directory to store the downloaded files
os.makedirs('price_data', exist_ok=True)

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find all anchor tags (<a>) that contain links to CSV files
links = soup.find_all('a', href=True)

# Extract the filenames from the href attributes of the anchor tags
csv_files = [link['href'] for link in links if link['href'].endswith('.csv')]

# Download each CSV file and save it in the "data" directory
for filename in csv_files:
    file_url = url + filename
    response = requests.get(file_url)
    with open(os.path.join('price_data', filename), 'wb') as f:
        f.write(response.content)
        print(f"Downloaded {filename}")



Downloaded PUB_PriceHOEPPredispOR.csv
Downloaded PUB_PriceHOEPPredispOR_2002.csv
Downloaded PUB_PriceHOEPPredispOR_2002_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2003.csv
Downloaded PUB_PriceHOEPPredispOR_2003_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2004.csv
Downloaded PUB_PriceHOEPPredispOR_2004_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2005.csv
Downloaded PUB_PriceHOEPPredispOR_2005_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2006.csv
Downloaded PUB_PriceHOEPPredispOR_2006_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2007.csv
Downloaded PUB_PriceHOEPPredispOR_2007_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2008.csv
Downloaded PUB_PriceHOEPPredispOR_2008_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2009.csv
Downloaded PUB_PriceHOEPPredispOR_2009_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2010.csv
Downloaded PUB_PriceHOEPPredispOR_2010_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2011.csv
Downloaded PUB_PriceHOEPPredispOR_2011_v1.csv
Downloaded PUB_PriceHOEPPredispOR_2012.csv
Downloaded PUB_PriceHOEPPredi

In [157]:
# Combine all CSV files in the "data" directory into a single DataFrame
df = combine_csvs('price_data')
# Combine the 'Date' and 'Hour' columns into a single 'DateTime' column
df = combine_date_hour(df)
## set index to DateTime
df.set_index('DateTime', inplace=True)
## Sort the DataFrame by the 'DateTime' column
df.sort_values(by='DateTime', inplace=True)
# Remove duplicate rows
df = df.drop_duplicates()
# Display the first 50 rows of the DataFrame
df.head(50)

,HOEP,Hour 1 Predispatch,Hour 2 Predispatch,Hour 3 Predispatch,OR 10 Min Sync,OR 10 Min non-sync,OR 30 Min
DateTime,,,,,,,
2002-05-01 00:00:00,22.97,14.53,14.53,14.53,NaN,NaN,NaN
2002-05-01 01:00:00,23.27,21.31,21.37,21.37,NaN,NaN,NaN
2002-05-01 02:00:00,24.54,25.45,21.38,21.38,NaN,NaN,NaN
2002-05-01 03:00:00,15.17,24.92,24.92,14.53,NaN,NaN,NaN
2002-05-01 04:00:00,23.59,27.38,27.38,27.4,NaN,NaN,NaN
2002-05-01 05:00:00,28.84,30.34,30.18,30.18,NaN,NaN,NaN
2002-05-01 06:00:00,30.63,30.78,30.78,30.49,NaN,NaN,NaN
2002-05-01 07:00:00,32.23,30.92,30.92,30.92,NaN,NaN,NaN
2002-05-01 08:00:00,30,30.22,30.73,30.73,NaN,NaN,NaN


#### Weather Data (Environment Canada)




In [149]:
## Test out getting 1 month of data 

month = "01" # January
year = "2020" # 2020
stationID = 51459 #Vancouver

base_url = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
query_url = "format=csv&stationID={}&Year={}&Month={}&timeframe=1".format(stationID, year, month)
api_endpoint = base_url + query_url

print("Click me to download CSV data:")
print(api_endpoint)

Click me to download CSV data:
http://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=51442&Year=2020&Month=01&timeframe=1


In [ ]:
month = "01" # January
year = "2020" # 2020
stationID = 51442 #Vancouver

base_url = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
query_url = "format=csv&stationID={}&Year={}&Month={}&timeframe=1".format(stationID, year, month)
api_endpoint = base_url + query_url

print("Click me to download CSV data:")
print(api_endpoint)

https://climate.weather.gc.ca/climate_data/daily_data_e.html?

### Ontario Holidays 



In [165]:
## Add holidays to the demand data frame
## Copy deamnd df 
df3 = df_d.copy()

In [167]:
import holidays 
hols = holidays.Canada(state = 'ON')


In [168]:
## Add holidays to the demand data frame

df3['Holiday'] = pd.Series(df3.index.date).apply(lambda x: x in hols).values

In [169]:
df3.head(50)

,Market Demand,Ontario Demand,Holiday
DateTime,,,
2002-05-01 00:00:00,14141,14137,False
2002-05-01 01:00:00,13876,13872,False
2002-05-01 02:00:00,13974,13820,False
2002-05-01 03:00:00,13898,13744,False
2002-05-01 04:00:00,14378,14224,False
2002-05-01 05:00:00,15408,15404,False
2002-05-01 06:00:00,17070,17066,False
2002-05-01 07:00:00,17644,17640,False
2002-05-01 08:00:00,17723,17719,False


## Population Data 



In [172]:


# URL of the webpage containing the links to the CSV files
url = 'https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100287/en'

# Create a directory to store the downloaded files
os.makedirs('population_data', exist_ok=True)

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find all anchor tags (<a>) that contain links to CSV files
links = soup.find_all('a', href=True)

# Extract the filenames from the href attributes of the anchor tags
csv_files = [link['href'] for link in links if link['href'].endswith('.csv')]

# Download each CSV file and save it in the "data" directory
for filename in csv_files:
    file_url = url + filename
    response = requests.get(file_url)
    with open(os.path.join('population_data', filename), 'wb') as f:
        f.write(response.content)
        print(f"Downloaded {filename}")


/var/folders/vr/r8mf2bqs5rq239fpzg2_n3z00000gn/T/ipykernel_3788/640124098.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.content, 'html.parser')


In [173]:
import os
import requests
from bs4 import BeautifulSoup

# URL of the webpage containing the links to the CSV files
url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100287/en"

# Create a directory to store the downloaded files
os.makedirs('population_data', exist_ok=True)

# Send a GET request to the URL
response = requests.get(url)

# Check the content type of the response
content_type = response.headers.get('content-type')
print("Content-Type:", content_type)

# Parse the HTML content of the webpage if it's HTML
if 'text/html' in content_type:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all anchor tags (<a>) that contain links to CSV files
    links = soup.find_all('a', href=True)

    # Extract the filenames from the href attributes of the anchor tags
    csv_files = [link['href'] for link in links if link['href'].endswith('.csv')]

    # Download each CSV file and save it in the "population_data" directory
    for filename in csv_files:
        file_url = url + filename
        response = requests.get(file_url)
        with open(os.path.join('population_data', filename), 'wb') as f:
            f.write(response.content)
            print(f"Downloaded {filename}")
else:
    # If the content type is not HTML, handle it accordingly (e.g., save it directly to a file)
    with open('population_data/full_table.csv', 'wb') as f:
        f.write(response.content)
        print("Downloaded full_table.csv")


Content-Type: application/json
Downloaded full_table.csv


In [175]:
import os
import requests

# URL of the webpage containing the links to the CSV files
url = 'https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100287/en'

# Create a directory to store the downloaded files
os.makedirs('population_data', exist_ok=True)

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content of the response
    json_data = response.json()
    
    # Extract the download link from the JSON data
    download_link = json_data['object']
    
    # Download the CSV file and save it in the "population_data" directory
    filename = 'full_table.csv'
    response = requests.get(download_link)
    if response.status_code == 200:
        with open(os.path.join('population_data', filename), 'wb') as f:
            f.write(response.content)
            print(f"Downloaded {filename}")
    else:
        print("Failed to download the CSV file.")
else:
    print("Failed to retrieve data from the URL.")


Downloaded full_table.csv


In [183]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website to scrape
url = 'https://www.neilsberg.com/insights/ontario-or-population-by-year/'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing population data
table = soup.find('table')

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
data = []
for row in table.find_all('tr')[1:]:
    row_data = [td.text.strip() for td in row.find_all('td')]
    data.append(row_data)

# Create a DataFrame
df = pd.DataFrame(data, columns=headers)

# Save the DataFrame as a CSV file
df.to_csv('ontario_population.csv', index=False)

print("CSV file saved successfully.")


CSV file saved successfully.


In [186]:
## Get the population data from the CSV file
df_pop = pd.read_csv('ontario_population.csv')
df_pop.head(30)

,Year,Population,Year on Year Change,Change in Percent
0,2000,"11,216",-,-
1,2001,"11,283",67,0.6%
2,2002,"11,278",-5,-0.04%
3,2003,"11,229",-49,-0.43%
4,2004,"11,178",-51,-0.45%
5,2005,"11,196",18,0.16%
6,2006,"11,199",3,0.03%
7,2007,"11,236",37,0.33%
8,2008,"11,152",-84,-0.75%
9,2009,"11,123",-29,-0.26%
